# ADS-508-01-SP23 Team 8: Final Project

# Train model

Much of the code is modified from `Fregly, C., & Barth, A. (2021). Data science on AWS: Implementing end-to-end, continuous AI and machine learning pipelines. O’Reilly.`

## Install missing dependencies

[PyAthena](https://pypi.org/project/PyAthena/) is a Python DB API 2.0 (PEP 249) compliant client for Amazon Athena.

In [2]:
!pip install --disable-pip-version-check -q PyAthena==2.1.0
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26
!pip install missingno

## Globally import libraries

In [16]:
import boto3
from botocore.client import ClientError
import pandas as pd
from pyathena import connect
from IPython.core.display import display, HTML
import missingno as msno
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error
import datetime as dt
import time
import sagemaker
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
import joblib
import os

## Instantiate AWS SageMaker and S3 sessions

In [4]:
session = boto3.session.Session()
role = sagemaker.get_execution_role()
region = session.region_name
sagemaker_session = sagemaker.Session()
def_bucket = sagemaker_session.default_bucket()
bucket = 'sagemaker-us-east-ads508-sp23-t8'

s3 = boto3.Session().client(service_name="s3",
                            region_name=region)

sm = boto3.Session().client(service_name="sagemaker",
                            region_name=region)

In [5]:
setup_s3_bucket_passed = False
ingest_create_athena_db_passed = False
ingest_create_athena_table_tsv_passed = False

In [6]:
print(f"Default bucket: {def_bucket}")
print(f"Public T8 bucket: {bucket}")

Default bucket: sagemaker-us-east-1-657724983756
Public T8 bucket: sagemaker-us-east-ads508-sp23-t8


## Verify S3 Bucket Creation

In [7]:
%%bash

aws s3 ls s3://${bucket}/

2023-03-16 17:05:02 aws-athena-query-results-657724983756-us-east-1
2023-03-02 16:56:48 sagemaker-studio-657724983756-5nh7ydsouq7
2023-03-02 17:25:41 sagemaker-studio-657724983756-7yc8bp8xk0b
2023-03-02 17:01:51 sagemaker-us-east-1-657724983756
2023-03-17 05:19:31 sagemaker-us-east-ads508-sp23-t8


In [8]:
response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print(f"[ERROR] Cannot find bucket {bucket} in {response} due to {e}.")

{'ResponseMetadata': {'RequestId': '9YK518SHN4V6RTVV', 'HostId': 'Le7sSDUyINFrS+FZLRZxfFb/Q3ElGn1s7TasI+ixJZbdQmXdEO9DC1CL8DnTY/VY6Z9GxmNsBpc=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Le7sSDUyINFrS+FZLRZxfFb/Q3ElGn1s7TasI+ixJZbdQmXdEO9DC1CL8DnTY/VY6Z9GxmNsBpc=', 'x-amz-request-id': '9YK518SHN4V6RTVV', 'date': 'Mon, 27 Mar 2023 23:14:47 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


In [9]:
%store setup_s3_bucket_passed

Stored 'setup_s3_bucket_passed' (bool)


## Pass in ABT from CSV

In [10]:
s3_abt_csv_path = f"s3://{def_bucket}/team_8_data/abt/abt_encoded_df01.csv"
abt_encoded_df01 = pd.read_csv(s3_abt_csv_path)

In [11]:
y01 = ['childpoverty']
abt_encoded_y01_vc01 = abt_encoded_df01[y01].to_numpy()
print(abt_encoded_y01_vc01.shape)
display(abt_encoded_y01_vc01[0:11])
abt_encoded_x01_df01 = abt_encoded_df01.drop(y01, axis=1)
print(abt_encoded_x01_df01.shape)
display(abt_encoded_x01_df01.head(11))

abt_encoded_x01_df01['boroughs'] = abt_encoded_x01_df01['borough_bronx'].astype(int).astype(str) + abt_encoded_x01_df01['borough_brooklyn'].astype(int).astype(str) + abt_encoded_x01_df01['borough_manhattan'].astype(int).astype(str) + abt_encoded_x01_df01['borough_queens'].astype(int).astype(str) + abt_encoded_x01_df01['borough_staten island'].astype(int).astype(str)
display(abt_encoded_x01_df01.head(5))
train_x01, test_x01, train_y01, test_y01 = train_test_split(abt_encoded_x01_df01,
                                                            abt_encoded_y01_vc01,
                                                            test_size=.2,
                                                            stratify=abt_encoded_x01_df01[['boroughs']],
                                                            shuffle=True,
                                                            random_state=1699)

train_x01 = train_x01.drop(['boroughs'], axis=1)
test_x01 = test_x01.drop(['boroughs'], axis=1)

train_y01 = train_y01.ravel()
test_y01 = test_y01.ravel()


print(f'{train_x01.shape}')
print(f'{train_y01.shape}')
print(f'\n{test_x01.shape}')
print(f'{test_y01.shape}')

(31605, 1)


array([[20.7],
       [23.6],
       [35.9],
       [31.5],
       [67.7],
       [68.3],
       [ 0. ],
       [62.4],
       [64.9],
       [63.1],
       [ 6.5]])

(31605, 49)


,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,relative_data_year_-4,relative_data_year_-3,relative_data_year_-2,relative_data_year_-1,relative_data_year_0,...,walk,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.9,0.0,0.0,43.0,2308.0,80.8,16.2,2.9,0.0,7.7
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.4,0.5,2.1,45.0,2675.0,71.7,25.3,2.5,0.6,9.5
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,8.6,1.6,1.7,38.8,2120.0,75.0,21.3,3.8,0.0,8.7
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.0,2.4,6.2,45.4,1083.0,76.8,15.5,7.7,0.0,19.2
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.3,1.0,0.0,46.0,2508.0,71.0,21.3,7.7,0.0,17.2
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,14.0,1.5,4.1,42.7,1191.0,74.2,16.1,9.7,0.0,18.9
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,113.0,62.8,37.2,0.0,0.0,0.0
7,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,17.7,1.8,2.7,35.5,1691.0,85.1,8.3,6.1,0.5,9.4
8,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,18.0,0.0,1.6,42.8,1102.0,86.9,8.5,4.5,0.0,15.2
9,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7.1,0.7,0.5,44.0,1559.0,75.0,14.0,11.0,0.0,10.6


,borough_bronx,borough_brooklyn,borough_manhattan,borough_queens,borough_staten island,relative_data_year_-4,relative_data_year_-3,relative_data_year_-2,relative_data_year_-1,relative_data_year_0,...,othertransp,workathome,meancommute,employed,privatework,publicwork,selfemployed,familywork,unemployment,boroughs
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,43.0,2308.0,80.8,16.2,2.9,0.0,7.7,10000
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.5,2.1,45.0,2675.0,71.7,25.3,2.5,0.6,9.5,10000
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.6,1.7,38.8,2120.0,75.0,21.3,3.8,0.0,8.7,10000
3,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.4,6.2,45.4,1083.0,76.8,15.5,7.7,0.0,19.2,10000
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,46.0,2508.0,71.0,21.3,7.7,0.0,17.2,10000


(25284, 49)
(25284,)

(6321, 49)
(6321,)


## Model Training using Grid search with 5-fold cross-validation

### Random Forests

In [12]:
# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m1v1_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                     ('ss', StandardScaler()),
                     ('rf', RandomForestRegressor(n_jobs=2, random_state=1699))])

#n_est_hparam = [1, 10, 100, 200]
#max_depth_hparam = [5, 10, None]
#min_samp_leaf_hparam = [1, 75, 105]
#max_feat_hparam = ['sqrt', 'log2']

n_est_hparam = [1, 10]
max_depth_hparam = [5, 10]
min_samp_leaf_hparam = [1, 75]
max_feat_hparam = ['sqrt']

m1v1_grd = {'rf__n_estimators': n_est_hparam,
            'rf__max_depth': max_depth_hparam,
            'rf__min_samples_leaf': min_samp_leaf_hparam,
            'rf__max_features': max_feat_hparam
           }

m1v1_rf = GridSearchCV(m1v1_pip,
                       m1v1_grd,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=2,
                       refit=True)

m1v1_rf.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m1v1_rf.best_estimator_}')

print(pd.DataFrame(m1v1_rf.cv_results_))

train_m1v1_y01_pred = m1v1_rf.predict(train_x01)
print(train_m1v1_y01_pred)

test_m1v1_y01_pred = m1v1_rf.predict(test_x01)
print(test_m1v1_y01_pred)

# Display evaluation metrics
# R-sq
train_m1v1_rf_r2 = r2_score(train_y01, train_m1v1_y01_pred)
test_m1v1_rf_r2 = r2_score(test_y01, test_m1v1_y01_pred)

print(f'Train R-sq:\n{train_m1v1_rf_r2}')
print(f'Test R-sq:\n{test_m1v1_rf_r2}')

# RMSE
train_m1v1_rf_rmse = mean_squared_error(train_y01, train_m1v1_y01_pred, squared=False)
test_m1v1_rf_rmse = mean_squared_error(test_y01, test_m1v1_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m1v1_rf_rmse}')
print(f'Test RMSE:\n{test_m1v1_rf_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

Best Estimator:
Pipeline(memory=None,
         steps=[('si',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('ss',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('rf',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                       criterion='mse', max_depth=10,
                                       max_features='sqrt', max_leaf_nodes=None,
                                       max_samples=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=10, n_jobs=2,


### Neural Network

In [13]:
# Start timer script
start_time = dt.datetime.today()

# Citation: Hochberg, 2018; Shanmukh, 2021
m2v1_pip = Pipeline([('si', SimpleImputer(strategy='median')),
                     ('ss', StandardScaler()),
                     ('nn', MLPRegressor(random_state=1699))])

nodes_h = 3
predictors_p = 49

#hidden_layer_sizes_hparam = [[100,],
#                             [(nodes_h*(predictors_p+1))+nodes_h+1,],
#                             [50, 50]
#                            ]
#activation_hparam = ['logistic', 'relu']
#solver_hparam = ['lbfgs', 'adam']
#alpha_hparam = [.0001, .0005, .001, .005]
#learn_rate_hparam = ['constant', 'invscaling']

hidden_layer_sizes_hparam = [[100,]]
activation_hparam = ['relu']
solver_hparam = ['adam']
alpha_hparam = [.0001]
learn_rate_hparam = ['invscaling']

m2v1_grd = {'nn__hidden_layer_sizes': hidden_layer_sizes_hparam,
            'nn__activation': activation_hparam,
            'nn__solver': solver_hparam,
            'nn__alpha': alpha_hparam,
            'nn__learning_rate': learn_rate_hparam
           }

m2v1_nn = GridSearchCV(m2v1_pip,
                       m2v1_grd,
                       scoring='neg_root_mean_squared_error',
                       n_jobs=2,
                       refit=True)

m2v1_nn.fit(train_x01, train_y01)

print(f'Best Estimator:\n{m2v1_nn.best_estimator_}')

print(pd.DataFrame(m2v1_nn.cv_results_))

train_m2v1_y01_pred = m2v1_nn.predict(train_x01)
print(train_m2v1_y01_pred)

test_m2v1_y01_pred = m2v1_nn.predict(test_x01)
print(test_m2v1_y01_pred)

# Display evaluation metrics
# R-sq
train_m2v1_nn_r2 = r2_score(train_y01, train_m2v1_y01_pred)
test_m2v1_nn_r2 = r2_score(test_y01, test_m2v1_y01_pred)

print(f'Train R-sq:\n{train_m2v1_nn_r2}')
print(f'Test R-sq:\n{test_m2v1_nn_r2}')

# RMSE
train_m2v1_nn_rmse = mean_squared_error(train_y01, train_m2v1_y01_pred, squared=False)
test_m2v1_nn_rmse = mean_squared_error(test_y01, test_m2v1_y01_pred, squared=False)

print(f'Train RMSE:\n{train_m2v1_nn_rmse}')
print(f'Test RMSE:\n{test_m2v1_nn_rmse}')

# End timer script
end_time = dt.datetime.today()
time_elapse = end_time - start_time
print(f'End Time = {end_time}')
print(f'Script Time = {time_elapse}')

Best Estimator:
Pipeline(memory=None,
         steps=[('si',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='median',
                               verbose=0)),
                ('ss',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('nn',
                 MLPRegressor(activation='relu', alpha=0.0001,
                              batch_size='auto', beta_1=0.9, beta_2=0.999,
                              early_stopping=False, epsilon=1e-08,
                              hidden_layer_sizes=[100],
                              learning_rate='invscaling',
                              learning_rate_init=0.001, max_fun=15000,
                              max_iter=200, momentum=0.9, n_iter_no_change=10,
                              nesterovs_momentum=True, power_t=0.5,
                              random_state=1699, shuffle=True, solver='adam',
        

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


## Model saving

In [19]:
s3_m1v1_rf_pqt_base_path = f"../models"

if not os.path.exists(s3_m1v1_rf_pqt_base_path):
    os.makedirs(s3_m1v1_rf_pqt_base_path)

s3_m1v1_rf_pqt_path = os.path.join(s3_m1v1_rf_pqt_base_path,
                                   'm1v1_rf.parquet')

# save the model to disk using joblib
joblib.dump(m1v1_rf,
            s3_m1v1_rf_pqt_path)

# load the saved model from disk using joblib
m1v1_rf_fitted = joblib.load(s3_m1v1_rf_pqt_path)

In [20]:
s3_m2v1_nn_pqt_base_path = f"../models"

if not os.path.exists(s3_m2v1_nn_pqt_base_path):
    os.makedirs(s3_m2v1_nn_pqt_base_path)

s3_m2v1_nn_pqt_path = os.path.join(s3_m2v1_nn_pqt_base_path,
                                   'm2v1_nn.parquet')

# save the model to disk using joblib
joblib.dump(m2v1_nn,
            s3_m2v1_nn_pqt_path)

# load the saved model from disk using joblib
m2v1_nn_fitted = joblib.load(s3_m2v1_nn_pqt_path)

## Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}